In [2]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
from utils import calculate_returns, compute_atr, compute_macd
import warnings
warnings.filterwarnings('ignore')

sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

master_ticker_data = sp500.copy()
master_ticker_data.rename(columns={"Symbol":"ticker"}, inplace = True)
master_ticker_data['ticker'] = master_ticker_data['ticker'].str.replace('.', '-')

sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')

symbols_list = sp500['Symbol'].unique().tolist()

end_date = '2023-12-31'

start_date = pd.to_datetime(end_date)-pd.DateOffset(365*10)

df = yf.download(tickers=symbols_list,
                 start=start_date,
                 end=end_date).stack()

df.index.names = ['date', 'ticker']

df.columns = df.columns.str.lower()

df

[*********************100%%**********************]  503 of 503 completed

4 Failed downloads:
['SOLV', 'SW', 'AMTM', 'GEV']: YFChartError("%ticker%: Data doesn't exist for startDate = 1388638800, endDate = 1703998800")


Price               adj close       close        high         low        open  \
date       ticker                                                               
2014-01-02 A        36.747047   40.207439   40.844063   40.164520   40.844063   
           AAPL     17.234299   19.754642   19.893929   19.715000   19.845715   
           ABBV     33.291317   51.980000   52.330002   51.520000   52.119999   
           ABT      30.904150   38.230000   38.400002   38.000000   38.090000   
           ACGL     19.123333   19.123333   19.850000   19.100000   19.753332   
...                       ...         ...         ...         ...         ...   
2023-12-29 XYL     113.428406  114.360001  114.680000  113.930000  114.089996   
           YUM     128.731308  130.660004  131.250000  130.210007  130.410004   
           ZBH     120.933395  121.699997  122.400002  121.239998  121.459999   
           ZBRA    273.329987  273.329987  276.309998  272.769989  274.730011   
           ZTS     195.434402  197.369995  198.009995  196.250000  196.679993   

Price                   volume  
date       ticker               
2014-01-02 A         2678848.0  
           AAPL    234684800.0  
           ABBV      4569100.0  
           ABT       4967500.0  
           ACGL      2592900.0  
...                        ...  
2023-12-29 XYL        698900.0  
           YUM       1196800.0  
           ZBH        849600.0  
           ZBRA       251300.0  
           ZTS       1007200.0  

[1220648 rows x 6 columns]

In [3]:
df['garman_klass_vol'] = ((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['adj close'])-np.log(df['open']))**2)

df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])
                                                          
df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])
                                                          
df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])

df['sharpe_ratio'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.sharpe_ratio(close=x))

df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(compute_macd)

df['dollar_volume'] = (df['adj close']*df['volume'])/1e6

df

Price               adj close       close        high         low        open  \
date       ticker                                                               
2014-01-02 A        36.747047   40.207439   40.844063   40.164520   40.844063   
           AAPL     17.234299   19.754642   19.893929   19.715000   19.845715   
           ABBV     33.291317   51.980000   52.330002   51.520000   52.119999   
           ABT      30.904150   38.230000   38.400002   38.000000   38.090000   
           ACGL     19.123333   19.123333   19.850000   19.100000   19.753332   
...                       ...         ...         ...         ...         ...   
2023-12-29 XYL     113.428406  114.360001  114.680000  113.930000  114.089996   
           YUM     128.731308  130.660004  131.250000  130.210007  130.410004   
           ZBH     120.933395  121.699997  122.400002  121.239998  121.459999   
           ZBRA    273.329987  273.329987  276.309998  272.769989  274.730011   
           ZTS     195.434402  197.369995  198.009995  196.250000  196.679993   

Price                   volume  garman_klass_vol        rsi    bb_low  \
date       ticker                                                       
2014-01-02 A         2678848.0         -0.004175        NaN       NaN   
           AAPL    234684800.0         -0.007649        NaN       NaN   
           ABBV      4569100.0         -0.077496        NaN       NaN   
           ABT       4967500.0         -0.016829        NaN       NaN   
           ACGL      2592900.0          0.000336        NaN       NaN   
...                        ...               ...        ...       ...   
2023-12-29 XYL        698900.0          0.000008  75.881272  4.644986   
           YUM       1196800.0         -0.000033  59.338419  4.809882   
           ZBH        849600.0          0.000038  68.767219  4.747299   
           ZBRA       251300.0          0.000073  67.914673  5.415905   
           ZTS       1007200.0          0.000024  66.791376  5.176648   

Price                bb_mid   bb_high  sharpe_ratio       atr      macd  \
date       ticker                                                         
2014-01-02 A            NaN       NaN      0.638107       NaN       NaN   
           AAPL         NaN       NaN      0.992655       NaN       NaN   
           ABBV         NaN       NaN      0.697838       NaN       NaN   
           ABT          NaN       NaN      0.652657       NaN       NaN   
           ACGL         NaN       NaN      0.654709       NaN       NaN   
...                     ...       ...           ...       ...       ...   
2023-12-29 XYL     4.699440  4.753893      0.632795 -0.071904  2.215117   
           YUM     4.848037  4.886192      0.565898  0.100551  0.893190   
           ZBH     4.778204  4.809110      0.278126 -0.907728  1.063465   
           ZBRA    5.546306  5.676707      0.614979  0.079474  1.761343   
           ZTS     5.249645  5.322641      0.861607  0.649087  2.432947   

Price              dollar_volume  
date       ticker                 
2014-01-02 A           98.439754  
           AAPL      4044.627945  
           ABBV       152.111356  
           ABT        153.516365  
           ACGL        49.584890  
...                          ...  
2023-12-29 XYL         79.275113  
           YUM        154.065629  
           ZBH        102.745013  
           ZBRA        68.687826  
           ZTS        196.841530  

[1220648 rows x 15 columns]

In [4]:
data = df.groupby(level=1, group_keys=False).apply(calculate_returns).dropna()

data

Price               adj close       close        high         low        open  \
date       ticker                                                               
2014-02-07 A        38.845566   42.503578   42.539341   41.580830   41.816879   
           AAPL     16.288992   18.559999   18.676071   18.477858   18.620714   
           ABBV     31.560284   48.889999   49.029999   47.660000   48.000000   
           ABT      30.223385   37.180000   37.209999   36.650002   36.770000   
           ACGL     17.583332   17.583332   17.663334   17.410000   17.603333   
...                       ...         ...         ...         ...         ...   
2023-12-29 XYL     113.428406  114.360001  114.680000  113.930000  114.089996   
           YUM     128.731308  130.660004  131.250000  130.210007  130.410004   
           ZBH     120.933395  121.699997  122.400002  121.239998  121.459999   
           ZBRA    273.329987  273.329987  276.309998  272.769989  274.730011   
           ZTS     195.434402  197.369995  198.009995  196.250000  196.679993   

Price                   volume  garman_klass_vol        rsi    bb_low  \
date       ticker                                                       
2014-02-07 A         2749586.0         -0.001839  56.320226  3.634578   
           AAPL    370280400.0         -0.006857  44.058787  2.785700   
           ABBV      6979500.0         -0.067514  46.153243  3.435299   
           ABT      12028000.0         -0.014735  46.146218  3.379105   
           ACGL      1496100.0          0.000104  30.310212  2.913167   
...                        ...               ...        ...       ...   
2023-12-29 XYL        698900.0          0.000008  75.881272  4.644986   
           YUM       1196800.0         -0.000033  59.338419  4.809882   
           ZBH        849600.0          0.000038  68.767219  4.747299   
           ZBRA       251300.0          0.000073  67.914673  5.415905   
           ZTS       1007200.0          0.000024  66.791376  5.176648   

Price                bb_mid  ...  sharpe_ratio       atr      macd  \
date       ticker            ...                                     
2014-02-07 A       3.677225  ...      0.638107 -0.813500 -0.263344   
           AAPL    2.861338  ...      0.992655 -0.929876 -0.511188   
           ABBV    3.481890  ...      0.697838 -0.934993 -0.613572   
           ABT     3.450995  ...      0.652657 -0.874355 -0.931539   
           ACGL    2.945952  ...      0.654709 -1.033083 -0.749499   
...                     ...  ...           ...       ...       ...   
2023-12-29 XYL     4.699440  ...      0.632795 -0.071904  2.215117   
           YUM     4.848037  ...      0.565898  0.100551  0.893190   
           ZBH     4.778204  ...      0.278126 -0.907728  1.063465   
           ZBRA    5.546306  ...      0.614979  0.079474  1.761343   
           ZTS     5.249645  ...      0.861607  0.649087  2.432947   

Price              dollar_volume  return_1d  return_2d  return_3d  return_6d  \
date       ticker                                                              
2014-02-07 A          106.809224   0.019386   0.017178   0.009198  -0.000252   
           AAPL      6031.494447   0.013990   0.009901   0.009090   0.007530   
           ABBV       220.275000   0.020029   0.012079   0.006492   0.001991   
           ABT        363.526873   0.014461   0.011352   0.008480   0.002944   
           ACGL        26.306423   0.003233  -0.000757  -0.001826  -0.004815   
...                          ...        ...        ...        ...        ...   
2023-12-29 XYL         79.275113   0.000350  -0.000087   0.002166   0.007004   
           YUM        154.065629   0.001073  -0.000688   0.001023   0.002082   
           ZBH        102.745013   0.000575   0.002350   0.000768   0.002680   
           ZBRA        68.687826  -0.007336  -0.004470  -0.002632   0.007331   
           ZTS        196.841530   0.001065   0.001193   0.003178   0.002333   

Price              return_9d  return_12d  
d

#### Download Fama-French Factors and Calculate Rolling Factor Betas.

We will introduce the Fama—French data to estimate the exposure of assets to common risk factors using linear regression.

The five Fama—French factors, namely market risk, size, value, operating profitability, and investment have been shown empirically to explain asset returns and are commonly used to assess the risk/return profile of portfolios. Hence, it is natural to include past factor exposures as financial features in models.

We can access the historical factor returns using the pandas-datareader and estimate historical exposures using the RollingOLS rolling linear regression.

In [6]:
factor_data = web.DataReader('F-F_Research_Data_5_Factors_2x3',
                               'famafrench',
                               start='2010')[0].drop('RF', axis=1)

factor_data.index = factor_data.index.to_timestamp()

factor_data = factor_data.resample('M').last().div(100)

factor_data.index.name = 'date'

factor_data = factor_data.join(data['return_1d']).sort_index()

factor_data

Mkt-RF     SMB     HML     RMW     CMA  return_1d
date       ticker                                                   
2014-02-28 A       0.0465  0.0013 -0.0031 -0.0021 -0.0044   0.010113
           AAPL    0.0465  0.0013 -0.0031 -0.0021 -0.0044  -0.002710
           ABBV    0.0465  0.0013 -0.0031 -0.0021 -0.0044   0.000000
           ABT     0.0465  0.0013 -0.0031 -0.0021 -0.0044  -0.000251
           ACGL    0.0465  0.0013 -0.0031 -0.0021 -0.0044   0.004115
...                   ...     ...     ...     ...     ...        ...
2023-11-30 XYL     0.0883 -0.0011  0.0166 -0.0381 -0.0099   0.015749
           YUM     0.0883 -0.0011  0.0166 -0.0381 -0.0099   0.000239
           ZBH     0.0883 -0.0011  0.0166 -0.0381 -0.0099   0.020980
           ZBRA    0.0883 -0.0011  0.0166 -0.0381 -0.0099   0.001521
           ZTS     0.0883 -0.0011  0.0166 -0.0381 -0.0099   0.005006

[40751 rows x 6 columns]

In [7]:
observations = factor_data.groupby(level=1).size()

valid_stocks = observations[observations >= 10]

factor_data = factor_data[factor_data.index.get_level_values('ticker').isin(valid_stocks.index)]

factor_data

Mkt-RF     SMB     HML     RMW     CMA  return_1d
date       ticker                                                   
2014-02-28 A       0.0465  0.0013 -0.0031 -0.0021 -0.0044   0.010113
           AAPL    0.0465  0.0013 -0.0031 -0.0021 -0.0044  -0.002710
           ABBV    0.0465  0.0013 -0.0031 -0.0021 -0.0044   0.000000
           ABT     0.0465  0.0013 -0.0031 -0.0021 -0.0044  -0.000251
           ACGL    0.0465  0.0013 -0.0031 -0.0021 -0.0044   0.004115
...                   ...     ...     ...     ...     ...        ...
2023-11-30 XYL     0.0883 -0.0011  0.0166 -0.0381 -0.0099   0.015749
           YUM     0.0883 -0.0011  0.0166 -0.0381 -0.0099   0.000239
           ZBH     0.0883 -0.0011  0.0166 -0.0381 -0.0099   0.020980
           ZBRA    0.0883 -0.0011  0.0166 -0.0381 -0.0099   0.001521
           ZTS     0.0883 -0.0011  0.0166 -0.0381 -0.0099   0.005006

[40736 rows x 6 columns]

Calculate Rolling Factor Betas.

In [9]:
betas = (factor_data.groupby(level=1,
                            group_keys=False)
         .apply(lambda x: RollingOLS(endog=x['return_1d'], 
                                     exog=sm.add_constant(x.drop('return_1d', axis=1)),
                                     window=min(24, x.shape[0]),
                                     min_nobs=len(x.columns)+1)
         .fit(params_only=True)
         .params
         .drop('const', axis=1)))

betas

Mkt-RF       SMB       HML       RMW       CMA
date       ticker                                                  
2014-02-28 A            NaN       NaN       NaN       NaN       NaN
           AAPL         NaN       NaN       NaN       NaN       NaN
           ABBV         NaN       NaN       NaN       NaN       NaN
           ABT          NaN       NaN       NaN       NaN       NaN
           ACGL         NaN       NaN       NaN       NaN       NaN
...                     ...       ...       ...       ...       ...
2023-11-30 XYL     0.080003 -0.255599  0.117233 -0.002383 -0.264587
           YUM     0.062013 -0.157398  0.142980 -0.001353 -0.160848
           ZBH     0.143010 -0.243773  0.098131 -0.175321 -0.105498
           ZBRA    0.094367 -0.105939  0.182311 -0.036620 -0.220842
           ZTS     0.051892 -0.172687  0.042099  0.021417 -0.041346

[40736 rows x 5 columns]

Join the rolling factors data to the main features dataframe

In [10]:
factors = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']

data = (data.join(betas.groupby('ticker').shift()))

data.loc[:, factors] = data.groupby('ticker', group_keys=False)[factors].apply(lambda x: x.fillna(x.mean()))

data = data.drop('adj close', axis=1)

data = data.dropna()

data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1207295 entries, (Timestamp('2014-02-07 00:00:00'), 'A') to (Timestamp('2023-12-29 00:00:00'), 'ZTS')
Data columns (total 25 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   close             1207295 non-null  float64
 1   high              1207295 non-null  float64
 2   low               1207295 non-null  float64
 3   open              1207295 non-null  float64
 4   volume            1207295 non-null  float64
 5   garman_klass_vol  1207295 non-null  float64
 6   rsi               1207295 non-null  float64
 7   bb_low            1207295 non-null  float64
 8   bb_mid            1207295 non-null  float64
 9   bb_high           1207295 non-null  float64
 10  sharpe_ratio      1207295 non-null  float64
 11  atr               1207295 non-null  float64
 12  macd              1207295 non-null  float64
 13  dollar_volume     1207295 non-null  float64
 14  return_1d         1207295 

In [11]:
data.head()

close       high        low       open       volume  \
date       ticker                                                            
2014-02-07 A       42.503578  42.539341  41.580830  41.816879    2749586.0   
           AAPL    18.559999  18.676071  18.477858  18.620714  370280400.0   
           ABBV    48.889999  49.029999  47.660000  48.000000    6979500.0   
           ABT     37.180000  37.209999  36.650002  36.770000   12028000.0   
           ACGL    17.583332  17.663334  17.410000  17.603333    1496100.0   

                   garman_klass_vol        rsi    bb_low    bb_mid   bb_high  \
date       ticker                                                              
2014-02-07 A              -0.001839  56.320226  3.634578  3.677225  3.719872   
           AAPL           -0.006857  44.058787  2.785700  2.861338  2.936976   
           ABBV           -0.067514  46.153243  3.435299  3.481890  3.528481   
           ABT            -0.014735  46.146218  3.379105  3.450995  3.522885   
           ACGL            0.000104  30.310212  2.913167  2.945952  2.978738   

                   ...  return_2d  return_3d  return_6d  return_9d  \
date       ticker  ...                                               
2014-02-07 A       ...   0.017178   0.009198  -0.000252   0.002117   
           AAPL    ...   0.009901   0.009090   0.007530  -0.005722   
           ABBV    ...   0.012079   0.006492   0.001991   0.004795   
           ABT     ...   0.011352   0.008480   0.002944   0.002665   
           ACGL    ...  -0.000757  -0.001826  -0.004815  -0.001795   

                   return_12d    Mkt-RF       SMB       HML       RMW  \
date       ticker                                                       
2014-02-07 A        -0.002089 -0.018428 -0.034321  0.027925  0.014862   
           AAPL     -0.004447  0.053902  0.008192 -0.033890  0.145374   
           ABBV     -0.000323  0.058951  0.079235 -0.097550  0.208368   
           ABT      -0.001969 -0.026242  0.059898 -0.027832  0.136349   
           ACGL     -0.003656 -0.064590 -0.034905  0.148707 -0.067339   

                        CMA  
date       ticker            
2014-02-07 A       0.026530  
           AAPL    0.193726  
           ABBV   -0.057851  
           ABT    -0.043127  
           ACGL   -0.136078  

[5 rows x 25 columns]

### Save price data

In [12]:
data.reset_index(inplace = True)

In [13]:
data.to_parquet("data/price_data.gzip", compression='gzip', index = False)
data.to_csv("data/price_data.csv", index = False)

In [14]:
data.head()

,date,ticker,close,high,low,open,volume,garman_klass_vol,rsi,bb_low,...,return_2d,return_3d,return_6d,return_9d,return_12d,Mkt-RF,SMB,HML,RMW,CMA
0,2014-02-07,A,42.503578,42.539341,41.580830,41.816879,2749586.0,-0.001839,56.320226,3.634578,...,0.017178,0.009198,-0.000252,0.002117,-0.002089,-0.018428,-0.034321,0.027925,0.014862,0.026530
1,2014-02-07,AAPL,18.559999,18.676071,18.477858,18.620714,370280400.0,-0.006857,44.058787,2.785700,...,0.009901,0.009090,0.007530,-0.005722,-0.004447,0.053902,0.008192,-0.033890,0.145374,0.193726
2,2014-02-07,ABBV,48.889999,49.029999,47.660000,48.000000,6979500.0,-0.067514,46.153243,3.435299,...,0.012079,0.006492,0.001991,0.004795,-0.000323,0.058951,0.079235,-0.097550,0.208368,-0.057851
3,2014-02-07,ABT,37.180000,37.209999,36.650002,36.770000,12028000.0,-0.014735,46.146218,3.379105,...,0.011352,0.008480,0.002944,0.002665,-0.001969,-0.026242,0.059898,-0.027832,0.136349,-0.043127
4,2014-02-07,ACGL,17.583332,17.663334,17.410000,17.603333,1496100.0,0.000104,30.310212,2.913167,...,-0.000757,-0.001826,-0.004815,-0.001795,-0.003656,-0.064590,-0.034905,0.148707,-0.067339,-0.136078


### Save data to sql db

In [15]:
import sqlite3
conn = sqlite3.connect('data/database.db')
c = conn.cursor()

In [16]:
data.to_sql('price_data', conn, if_exists='replace', index = False)
master_ticker_data.to_sql('master_ticker', conn, if_exists='replace')

503